In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
import os
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [22]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [3]:
%run ./Combine_csv.ipynb

In [23]:
prefix_dir ='Combined CSV/'
suffix_dir = 'combined_sports-'+Today_date+'.csv'
today_csv=os.path.join(prefix_dir+suffix_dir)

In [24]:
sports = pd.read_csv(today_csv)
sports.head()

,Source,Heading,Category,Date,Time,URL
0,Hindiustan Times,"ICC World Cup 2019: South Africa, Afghanistan ...",cricket,2019-06-14,19:23:00,https://www.hindustantimes.com/cricket/icc-wor...
1,Hindiustan Times,"ICC World Cup 2019: Beware of David Warner, he...",cricket,2019-06-14,14:59:00,https://www.hindustantimes.com/cricket/icc-wor...
2,Hindiustan Times,"India vs Pakistan, World Cup 2019: Will not be...",cricket,2019-06-14,11:01:00,https://www.hindustantimes.com/cricket/india-v...
3,Hindiustan Times,"ICC World Cup 2019, South Africa vs Afghanista...",cricket,2019-06-14,21:54:00,https://www.hindustantimes.com/cricket/icc-wor...
4,Hindiustan Times,ICC World Cup 2019: Australia fret on team bal...,cricket,2019-06-14,20:59:00,https://www.hindustantimes.com/cricket/icc-wor...


In [25]:
sports.shape

(834, 6)

In [26]:
df=sports
Heading = df['Heading']

In [27]:
df['Category'].value_counts()

others               168
tennis               111
cricket              109
football             106
hockey                59
athletics             56
snooker-billiards     25
cycling               25
boxing                25
wrestling             25
icc wc 2019           25
racing                25
golf                  25
shooting              25
chess                 25
Name: Category, dtype: int64

### Tokenizing and Stemming

In [28]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
from nltk.stem.snowball import SnowballStemmer


In [29]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
stemmer = SnowballStemmer("english")
stopset = set(stopwords.words('english'))

In [31]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [32]:
df['Heading'] = df['Heading'].apply(remove_apostrophe)
df['Heading'] = df['Heading'].apply(remove_punctuation)
df['Heading'] = df['Heading'].apply(convert_numbers)

In [33]:
def tokenize_and_stem(text):
   
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):

    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
   
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [34]:
Heading_stemmed = []
Heading_tokenized = []
for i in Heading:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'Heading', tokenize/stem
    Heading_stemmed.extend(allwords_stemmed) #extend the 'Heading_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    Heading_tokenized.extend(allwords_tokenized)

In [35]:
words_frame = pd.DataFrame({'words': Heading_tokenized}, index = Heading_stemmed)

In [36]:
words_frame.head(10)

,words
icc,icc
world,world
cup,cup
south,south
africa,africa
afghanistan,afghanistan
eye,eye
first,first
win,win
of,of


### Spliting and applying algorithms

In [37]:
def get_best_n(tfidf_matrix,dist):
    
    n_clusters = list (range (14,20))
    min=999
    for n in n_clusters:
        km_ss = KMeans(n_clusters=n)
        clusters_ss = km_ss.fit_predict(tfidf_matrix)
        score = silhouette_score(dist,clusters_ss)
        if min>score:
            min=score
            n_score=n
    return n_score

In [38]:
df_dict={}
for cat in df['Category'].unique():
    temp = df[df['Category']==cat].reset_index().drop(['index'],axis=1)
    Heading = temp['Heading']
    vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,4))
    tfidf_matrix = vectorizer.fit_transform(Heading)
    
    dist = cosine_similarity(tfidf_matrix)

    n = get_best_n(tfidf_matrix,dist)
    km = KMeans(n_clusters=n)
    km.fit(tfidf_matrix)
    
    clusters = km.labels_.tolist()
    temp['Cluster'] = clusters
 
    df_sorted=temp.sort_values(by='Cluster').reset_index()
    df_sorted.drop(['index'],axis=1,inplace=True)
    
    grp = df_sorted.sort_values('Cluster').groupby(['Cluster'],as_index=False)
    
    
    
    cluster_similarity_value =[]
    

    vectorizer = TfidfVectorizer()
    for i in range(n):
        group = grp.get_group(i)

        cluster_heading=group['Heading']
        cluster_matrix = vectorizer.fit_transform(cluster_heading)
        cluster_dist = cosine_similarity(cluster_matrix)
        cluster_elements_count = pd.DataFrame.count(group)
        x=[]
        for i in cluster_dist:

            if((cluster_elements_count[0]-1)==0):
                y=1
            else:
                y=float("{0:.2f}".format(((i.sum())/(cluster_elements_count[0]))))
            x.append(y)
            cluster_similarity_value.append(y)
   
    
    
    df_sorted['cluster_similarity_value']=cluster_similarity_value;  col=df_sorted.columns
    
    grp = df_sorted.sort_values('Cluster').groupby(['Cluster'],as_index=False)
    
    temp_more =[]
    temp_less  =[]
    for i in range(n):
        cluster = grp.get_group(i)
        cluster_mean = cluster['cluster_similarity_value'].mean()
        cluster_std = cluster['cluster_similarity_value'].std()
        comp_fact = cluster_mean + cluster_std/4
        for i in range(len(cluster)):
            if (cluster.iloc[i]['cluster_similarity_value']<comp_fact):
                temp_less.append(cluster.iloc[i])
            else:
                temp_more.append(cluster.iloc[i])
    df_more_similar=pd.DataFrame(temp_more,columns=col)
    df_less_similar=pd.DataFrame(temp_less,columns=col)
    
    
    Result = df_more_similar.sort_values('Cluster').groupby(['Cluster'],as_index=False).apply(lambda x: x.sample(1)) 
    Result = Result.reset_index().drop(['level_0','level_1'],axis=1)
    Result = Result.append(df_less_similar)
    Result = Result.sort_values(by='Cluster')
    Result = Result.reset_index().drop(['index'],axis=1)
    
    
    df_dict[cat] = Result
    

In [39]:
for typ,data in df_dict.items():
    
    outname =typ +'.csv'
    root = 'Categorized Output/'
    if not os.path.exists(root):
        os.mkdir(root)
    date_today= Today_date +'/'
    outdir=root+ date_today[:-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    df.to_csv(fullname,index=False,encoding='utf-8')
